Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [5]:
#from random import choices, random, choice
import random
import copy
from tqdm import tqdm
import lab9_lib

In [60]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = random.choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

01010100111011111000010100111001001001001101010101: 15.34%
01111001000001101010101100100010011010101110111000: 23.34%
00111000001011010111100100001101011011100101010010: 9.16%
01010110000011101011000011110110100100110111000010: 35.56%
01100010110001010011001011000001100100001010100100: 7.36%
10100101001110110000011000010010010110111111000110: 15.33%
00000100001111110100011101101001000111100101110111: 9.13%
10011010110001101101000111010000101101100011001111: 23.33%
01011101001111100101111001101000111010000100011000: 7.33%
01001111001110001000111011011101110000110100101000: 9.33%
10


In [8]:
class Genotype:

  def __init__(self, genome, fitness):
    self.genome = genome
    self.fitness = fitness



In [16]:
GENES = 1000
POP_SIZE = 20
OFF_SIZE = 20
TOURN_SIZE = 5
EPO_SIZE = 10000
MUTATION_PROB = 0.5

ESP_MUT = 0.05
ESP_MUT_FACT = 2
MAX_NUM_MUT = 50
MAX_NUM_MUT_ADD = 1

best_ind = None

def mutate1(individual, epo_mut):
  my_list = copy.deepcopy(individual.genome)
  for i in range(len(my_list)):
        if random.random() < epo_mut:
            my_list[i] = not my_list[i]
  return my_list

def mutate2(individual, epo_mut):
  my_list = copy.deepcopy(individual.genome)
  if random.random() < 0.9:
    num_mutations = random.choice(range(epo_mut)) + 1
  else:
    num_mutations = 1
  for _ in range(num_mutations):
    i = random.choice(range(len(my_list)))
    my_list[i] = not my_list[i]
  return my_list

mutate = mutate2

def xover(ind1, ind2):
  offspring_genome = []

  for gene1, gene2 in zip(ind1.genome, ind2.genome):
      if random.random() < 0.5:
          offspring_genome.append(gene2)
      else:
          offspring_genome.append(gene1)
  return offspring_genome

pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE)]
pop = [Genotype(val, fitness(val)) for val in pop_val]
for epo in tqdm(range(EPO_SIZE)):
  tourn = [random.choice(pop) for _ in range(TOURN_SIZE)]
  parent = max(tourn, key=lambda x : x.fitness)

  #epo_mut = ESP_MUT
  epo_mut = MAX_NUM_MUT

  for off in range(OFF_SIZE):
    child = copy.deepcopy(parent)

    if random.random() < MUTATION_PROB:
      child.genome = mutate(child, epo_mut)
    else:
      child.genome = xover(child, random.choice(tourn))

    child.fitness = fitness(child.genome)

    if best_ind is None or child.fitness > best_ind.fitness:
      best_ind = child

    worst = min(tourn, key=lambda x : x.fitness)
    if child.fitness > worst.fitness:
      #epo_mut = max(0.0001,epo_mut / ESP_MUT_FACT)
      epo_mut = max(1,epo_mut - MAX_NUM_MUT_ADD)
    else:
      #epo_mut = min(1,epo_mut * ESP_MUT_FACT)
      epo_mut = min(POP_SIZE,epo_mut + MAX_NUM_MUT_ADD)

    pop.append(child)
    pop = sorted(pop, key=lambda x: x.fitness, reverse=True)
    pop = pop[:POP_SIZE]


100%|██████████| 10000/10000 [03:42<00:00, 44.90it/s]


In [17]:
print(best_ind.fitness)

0.848


In [ ]:
#We have an high dimensional problem, but in binary domain: this case is the opposite of having a monodimensional problem in continuous domain
#Here is clear more than ever that my mutation step has to take into consideration both mesas and spikes:
#can really I say that I won't find a better solution because I don't mutate enough? Couldn't it be that I'm mutating too much too?
#

#I also need to distinguish among different regions of my search space, reguarding the mutation step, so I'm gonna save the one that has created an individual
#into it and I'll use it as a more accurate prior for its mutation

In [22]:
class Genotype:

  def __init__(self, genome, fitness, mutation):
    self.genome = genome
    self.fitness = fitness
    self.mutation = mutation


In [23]:
GENES = 1000
POP_SIZE = 20
OFF_SIZE = 20
TOURN_SIZE = 5
EPO_SIZE = 10000
MUTATION_PROB = 0.5

ESP_MUT = 0.05
ESP_MUT_FACT = 2
MAX_NUM_MUT = 50
MAX_NUM_MUT_ADD = 1

best_ind = None

# def mutate1(individual, epo_mut):
#   my_list = copy.deepcopy(individual.genome)
#   for i in range(len(my_list)):
#         if random.random() < epo_mut:
#             my_list[i] = not my_list[i]
#   return my_list

def mutate2(individual, epo_mut):
  my_list = copy.deepcopy(individual.genome)
  if random.random() < 0.9:
    num_mutations = random.choice(range(epo_mut)) + 1
  else:
    num_mutations = 1
  for _ in range(num_mutations):
    i = random.choice(range(len(my_list)))
    my_list[i] = not my_list[i]
  return my_list

mutate = mutate2

def xover(ind1, ind2):
  offspring_genome = []

  for gene1, gene2 in zip(ind1.genome, ind2.genome):
      if random.random() < 0.5:
          offspring_genome.append(gene2)
      else:
          offspring_genome.append(gene1)
  return offspring_genome

pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE)]
pop = [Genotype(val, fitness(val), MAX_NUM_MUT) for val in pop_val]
for epo in tqdm(range(EPO_SIZE)):
  tourn = [random.choice(pop) for _ in range(TOURN_SIZE)]
  parent = max(tourn, key=lambda x : x.fitness)

  #epo_mut = ESP_MUT
  epo_mut = parent.mutation

  for off in range(OFF_SIZE):
    child = copy.deepcopy(parent)

    if random.random() < MUTATION_PROB:
      child.genome = mutate(child, epo_mut)
    else:
      child.genome = xover(child, random.choice(tourn))

    child.mutation = epo_mut
    child.fitness = fitness(child.genome)

    if best_ind is None or child.fitness > best_ind.fitness:
      best_ind = child

    worst = min(tourn, key=lambda x : x.fitness)
    if child.fitness > worst.fitness:
      #epo_mut = max(0.0001,epo_mut / ESP_MUT_FACT)
      epo_mut = max(1,epo_mut - MAX_NUM_MUT_ADD)
    else:
      #epo_mut = min(1,epo_mut * ESP_MUT_FACT)
      epo_mut = min(POP_SIZE,epo_mut + MAX_NUM_MUT_ADD)

    pop.append(child)
    pop = sorted(pop, key=lambda x: x.fitness, reverse=True)
    pop = pop[:POP_SIZE]


100%|██████████| 10000/10000 [03:40<00:00, 45.33it/s]


In [24]:
print(best_ind.fitness)

0.91


In [ ]:
#We are getting better: what if I update the parent's mutation if no offspring does better?

In [25]:
class Genotype:

  def __init__(self, genome, fitness, mutation):
    self.genome = genome
    self.fitness = fitness
    self.mutation = mutation


In [38]:
GENES = 1000
POP_SIZE = 20
OFF_SIZE = 20
TOURN_SIZE = 5
EPO_SIZE = 10000
MUTATION_PROB = 0.8

ESP_MUT = 0.05
ESP_MUT_FACT = 2
MAX_NUM_MUT = 50
MAX_NUM_MUT_ADD = 1

best_ind = None

# def mutate1(individual, epo_mut):
#   my_list = copy.deepcopy(individual.genome)
#   for i in range(len(my_list)):
#         if random.random() < epo_mut:
#             my_list[i] = not my_list[i]
#   return my_list

def mutate2(individual, epo_mut):
  my_list = copy.deepcopy(individual.genome)
  if random.random() < 0.9:
    num_mutations = random.choice(range(epo_mut)) + 1
  else:
    num_mutations = random.choice(range(2)) + 1
  for _ in range(num_mutations):
    i = random.choice(range(len(my_list)))
    my_list[i] = not my_list[i]
  return my_list

mutate = mutate2

def xover(ind1, ind2):
  offspring_genome = []

  for gene1, gene2 in zip(ind1.genome, ind2.genome):
      if random.random() < 0.5:
          offspring_genome.append(gene2)
      else:
          offspring_genome.append(gene1)
  return offspring_genome

pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE)]
pop = [Genotype(val, fitness(val), MAX_NUM_MUT) for val in pop_val]
for epo in tqdm(range(EPO_SIZE)):
  tourn = [random.choice(pop) for _ in range(TOURN_SIZE)]
  parent = max(tourn, key=lambda x : x.fitness)

  #epo_mut = ESP_MUT
  epo_mut = parent.mutation
  child_made_it = False

  for off in range(OFF_SIZE):
    child = copy.deepcopy(parent)

    if random.random() < MUTATION_PROB:
      child.genome = mutate(child, epo_mut)
    else:
      child.genome = xover(child, random.choice(tourn))

    child.mutation = epo_mut
    child.fitness = fitness(child.genome)

    if best_ind is None or child.fitness > best_ind.fitness:
      best_ind = child

    worst = min(tourn, key=lambda x : x.fitness)
    if child.fitness > worst.fitness:
      #epo_mut = max(0.0001,epo_mut / ESP_MUT_FACT)
      epo_mut = max(1,epo_mut - MAX_NUM_MUT_ADD)
    else:
      #epo_mut = min(1,epo_mut * ESP_MUT_FACT)
      epo_mut = min(POP_SIZE,epo_mut + MAX_NUM_MUT_ADD)

    pop.append(child)
    pop = sorted(pop, key=lambda x: x.fitness, reverse=True)
    pop = pop[:POP_SIZE]
    if child in pop: child_made_it = True

  if not child_made_it: parent.mutation = epo_mut

100%|██████████| 10000/10000 [04:02<00:00, 41.24it/s]


In [39]:
print(best_ind.fitness)

0.958


In [ ]:
#goin better but I should find a way to generalize and both find a precise value for my brand new alternative mutation

#I will try extinction

In [40]:
class Genotype:

  def __init__(self, genome, fitness, mutation):
    self.genome = genome
    self.fitness = fitness
    self.mutation = mutation


In [69]:
GENES = 1000
POP_SIZE = 20
OFF_SIZE = 20
SVR_SIZE = 10
TOURN_SIZE = 5
EPO_SIZE = 10000
MUTATION_PROB = 0.8

ESP_MUT = 0.05
ESP_MUT_FACT = 2
MAX_NUM_MUT = 50
MAX_NUM_MUT_ADD = 1

best_ind = None

# def mutate1(individual, epo_mut):
#   my_list = copy.deepcopy(individual.genome)
#   for i in range(len(my_list)):
#         if random.random() < epo_mut:
#             my_list[i] = not my_list[i]
#   return my_list

def mutate2(individual, epo_mut):
  my_list = copy.deepcopy(individual.genome)
  if random.random() < 0.9:
    num_mutations = random.choice(range(epo_mut)) + 1
  else:
    num_mutations = random.choice(range(2)) + 1
  for _ in range(num_mutations):
    i = random.choice(range(len(my_list)))
    my_list[i] = not my_list[i]
  return my_list

mutate = mutate2

def xover1(ind1, ind2):
  offspring_genome = []

  for gene1, gene2 in zip(ind1.genome, ind2.genome):
      if random.random() < 0.5:
          offspring_genome.append(gene2)
      else:
          offspring_genome.append(gene1)
  return offspring_genome

def xover2(ind1, ind2):
  offspring_genome = []

  if random.random() < 0.4:
    for gene1, gene2 in zip(ind1.genome, ind2.genome):
        if random.random() < 0.5:
            offspring_genome.append(gene2)
        else:
            offspring_genome.append(gene1)
  else:
    cut_pos = [random.choice(range(GENES)) for _ in range(5)]
    get_ind1 = True
    for i, (gene1, gene2) in enumerate(zip(ind1.genome, ind2.genome)):
      if i in cut_pos: get_ind1 = not get_ind1
      if get_ind1:
        offspring_genome.append(gene1)
      else:
        offspring_genome.append(gene2)

  return offspring_genome

xover = xover2

pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE)]
pop = [Genotype(val, fitness(val), MAX_NUM_MUT) for val in pop_val]
for epo in tqdm(range(EPO_SIZE)):
  tourn = [random.choice(pop) for _ in range(TOURN_SIZE)]
  parent = max(tourn, key=lambda x : x.fitness)

  #epo_mut = ESP_MUT
  epo_mut = parent.mutation
  child_made_it = False

  for off in range(OFF_SIZE):
    child = copy.deepcopy(parent)

    if random.random() < MUTATION_PROB:
      child.genome = mutate(child, epo_mut)
    else:
      child.genome = xover(child, random.choice(tourn))

    child.mutation = epo_mut
    child.fitness = fitness(child.genome)

    if best_ind is None or child.fitness > best_ind.fitness:
      best_ind = child

    worst = min(tourn, key=lambda x : x.fitness)
    if child.fitness > worst.fitness:
      #epo_mut = max(0.0001,epo_mut / ESP_MUT_FACT)
      epo_mut = max(1,epo_mut - MAX_NUM_MUT_ADD)
    else:
      #epo_mut = min(1,epo_mut * ESP_MUT_FACT)
      epo_mut = min(POP_SIZE,epo_mut + MAX_NUM_MUT_ADD)

    pop.append(child)
    pop = sorted(pop, key=lambda x: x.fitness, reverse=True)
    pop = pop[:POP_SIZE]
    if child in pop: child_made_it = True

  if not child_made_it: parent.mutation = epo_mut
  if epo % 80 == 0:
    pop = pop[:SVR_SIZE]
    new_pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE - SVR_SIZE)]
    pop = pop + [Genotype(val, fitness(val), MAX_NUM_MUT) for val in new_pop_val]


100%|██████████| 10000/10000 [04:09<00:00, 40.10it/s]


In [70]:
print(best_ind.fitness)

0.3399


In [47]:
#Well, extinction is doin its job
#Also adding an n cut xover helped

#I still have to blacklist all my already created individuals: it will decrease my fitness calls and it matters just because it is part of my metrics

#Then maybe I can try to see what happens if I suppose that my fitness is monotonic

#Now I will try to make the number of cuts variable: dunno if increasing or decreasing it will help but I'll try to decrease it over epochs

In [124]:
#I will copy this 'cause fitness.calls has to be reset
fitness = lab9_lib.make_problem(10)
print(fitness.calls)

0


In [125]:
class Genotype:

  def __init__(self, genome, fitness, mutation):
    self.genome = genome
    self.fitness = fitness
    self.mutation = mutation


In [133]:
GENES = 1000
POP_SIZE = 20
OFF_SIZE = 30
TOURN_SIZE = 5

SVR_SIZE = 10
RAGNAROCK = 80

EPO_SIZE = 10000
MUTATION_PROB = 0.8

MAX_NUM_MUT = 50
MAX_NUM_MUT_ADD = 1

best_ind = None
blacklist = {}

def mutate2(individual, epo_mut):
  my_list = copy.deepcopy(individual.genome)
  if random.random() < 0.9:
    num_mutations = random.choice(range(epo_mut)) + 1
  else:
    num_mutations = random.choice(range(2)) + 1
  for _ in range(num_mutations):
    i = random.choice(range(len(my_list)))
    my_list[i] = 1 - my_list[i]
  return my_list

mutate = mutate2

def xover2(ind1, ind2):
  offspring_genome = []

  if random.random() < 0.2:
    for gene1, gene2 in zip(ind1.genome, ind2.genome):
        if random.random() < 0.5:
            offspring_genome.append(gene2)
        else:
            offspring_genome.append(gene1)
  else:
    cut_pos = [random.choice(range(GENES)) for _ in range(5)]
    get_ind1 = True
    for i, (gene1, gene2) in enumerate(zip(ind1.genome, ind2.genome)):
      if i in cut_pos: get_ind1 = not get_ind1
      if get_ind1:
        offspring_genome.append(gene1)
      else:
        offspring_genome.append(gene2)

  return offspring_genome

xover = xover2

pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE)]
pop = [Genotype(val, fitness(val), MAX_NUM_MUT) for val in pop_val]
for epo in tqdm(range(EPO_SIZE)):
  tourn = [random.choice(pop) for _ in range(TOURN_SIZE)]
  parent = max(tourn, key=lambda x : x.fitness)

  epo_mut = parent.mutation
  child_made_it = False

  for off in range(OFF_SIZE):
    child = copy.deepcopy(parent)

    if random.random() < MUTATION_PROB:
      child.genome = mutate(child, epo_mut)
    else:
      child.genome = xover(child, random.choice(tourn))

    child.mutation = epo_mut
    child_genome_tuple = tuple(child.genome)
    if child_genome_tuple not in blacklist:
      blacklist[child_genome_tuple] = child
      child.fitness = fitness(child.genome)

      if best_ind is None or child.fitness > best_ind.fitness:
        best_ind = child

      pop.append(child)
      pop = sorted(pop, key=lambda x: x.fitness, reverse=True)
      pop = pop[:POP_SIZE]
      if child in pop: child_made_it = True

    worst = min(tourn, key=lambda x : x.fitness)
    if child_genome_tuple not in blacklist and child.fitness > worst.fitness:
      epo_mut = max(1,epo_mut - MAX_NUM_MUT_ADD)
    else:
      epo_mut = min(POP_SIZE,epo_mut + MAX_NUM_MUT_ADD)

  if not child_made_it: parent.mutation = epo_mut
  if epo % RAGNAROCK == 0:
    pop = pop[:SVR_SIZE]
    new_pop_val = [random.choices([0,1], k=GENES) for _ in range(POP_SIZE - SVR_SIZE)]
    pop = pop + [Genotype(val, fitness(val), MAX_NUM_MUT) for val in new_pop_val]


100%|██████████| 10000/10000 [06:29<00:00, 25.70it/s]


In [134]:
print(best_ind.fitness, fitness.calls, best_ind.genome)

0.335 772846 [0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,